## Wczytywanie danych
Paczka Sklearn (Scikit Learn) to kolejne bardzo popularne narzędzie do uczenia maszynowego. Posiada bardzo przejrzyste API i spore wsparcie (scikit-learn.org/). W dzisiejszych zadaniach skupimy się na ładowaniu zbiorów danych, ich transformacji oraz algorytmach klasyfikacji. Podczas dzisiejszych laboratoriów wykorzystamy:
<ul>
    <li>NLTK - udostępniające metody prostego przetwarzania tekstu (tokenizacja, lematyzacja, stemming)</li>
    <li>Sklearn - paczkę do uczenia maszynowego</li>
    <li>Pandas - bibliotekę do wczytywania i obsługi zbiorów danych</li>
</ul>
<span style="color: #ff0000">Ponieważ część kodu jest już stworzona, w każdym zadaniu wyszczególnione są numery linii, w których należy wprowadzić modyfikacje, aby rozwiązać zadanie. Jeśli nie widzisz numeracji linii w kodzie w otwartym notebooku - możesz włączyć tę funkcjonalność poprzez wybór View -> toggle line numbers w górnym menu.</span><br/><br/>
Najpierw wczytajmy dane tekstowe ze zbioru, w którym posiadamy zestaw wiadomości e-mail oznaczonych jako spamowe lub niespamowe.
Ponieważ będziemy rozwiązywać problem klasyfikacji, oddzielamy dane do trenowania klasyfikatora oraz do weryfikacji jego jakości.
<br/>

<strong>Przeanalizuj i uruchom poniższy fragment kodu.</strong> Załaduje on odpowiednie dane do dwóch obiektów:
<ol>
<li>train: zbiór treningowy - dokumenty na których nauczymy klasyfikator</li>
<li>test: zbiór testowy - dokumenty na których przetestujemy klasyfikator</li>
</ol>

In [1]:
import pandas
import numpy as np

# ---------------- Ładowanie danych i oddzielanie zbioru treningowego od testowego ------

try:
    full_dataset = pandas.read_csv('spam_emails.csv', encoding='utf-8')      # wczytaj dane z pliku CSV
except:
    import s3fs
    full_dataset = pandas.read_csv("https://dwisniewski-put-pjn.s3.eu-north-1.amazonaws.com/spam_emails.csv")
full_dataset['label_num'] = full_dataset.label.map({'ham':0, 'spam':1})  # ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam", wykonujemy mapowanie tych wartości na liczby, co będzie potrzebne do wykonania klasyfikacji.

np.random.seed(0)                                       # ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
train_indices = np.random.rand(len(full_dataset)) < 0.7 # wylosuj 70% danych, które stworzą zbiór treningowy. train_indices, to wektor o długości liczności wczytanego zbioru danych, w którym każda pozycja (przykład) może przyjąć dwie wartości: 1.0 - wybierz do zbioru treningowego; 0.0 - wybierz do zbioru testowego

train = full_dataset[train_indices] # wybierz zbior treningowy (70%)
test = full_dataset[~train_indices] # wybierz zbiór testowy (dopełnienie treningowego - 30%)



# ---------------- Wyświetlanie statystyk -----------------


print("Elementów w zbiorze treningowym: {train}, testowym: {test}".format(
    train=len(train), test=len(test)
))

print("\n\nLiczność klas w zbiorze treningowym: ")
print(train.label.value_counts())  # wyświetl rozkład etykiet w kolumnie "label"

print("\n\nLiczność klas w zbiorze testowym: ")
print(test.label.value_counts())   # wyświetl rozkład etykiet w kolumnie "label"



full_dataset.head()                # wyświetl próbkę danych


Elementów w zbiorze treningowym: 1624, testowym: 733


Liczność klas w zbiorze treningowym: 
ham     1111
spam     513
Name: label, dtype: int64


Liczność klas w zbiorze testowym: 
ham     517
spam    216
Name: label, dtype: int64


,label,text,label_num
0,ham,Re: What to choose for Core i5 64 bits?>>> If ...,0
1,spam,"Strictly Private.Gooday, With warm heart my fr...",1
2,ham,"Re: Flash is open?On Sat, 15 May 2010 00:27:32...",0
3,ham,Re: Alsa/Redhat 8 compatabilityMatthias Saou (...,0
4,spam,"Hey hibody, Save 80% today Lixi Eights followi...",1


## Transformacja danych
Aby zastosować większość algorytmów uczenia maszynowego - dane wejściowe muszą być reprezentowane jako wektory liczb. Wykorzystajmy zatem narzędzia dostarczone przez Scikit-learn do tego celu. Użyjmy klasy **CountVectorizer()**, aby podzielić poszczególne dokumenty na słowa, a następnie stworzyć reprezentację "bag of words"
Dla przypomnienia - "bag of words" tworzony jest w nastepujący sposób:
<ol>
<li>Przeglądamy wszystkie dostępne dokumenty i tworzymy listę wszystkich unikalnych słów jakie napotkaliśmy (słownik).</li>
<li>Stworzona lista wyznacza nam wektor cech - każda pozycja w takim wektorze oznacza jedno z napotkanych słów.</li>
<li>Każdy z dokumentów mapowany jest na wektor cech poprzez zapisanie ile razy każde ze słów dokumentu wystąpiło w nim.</li>
</ol>
Przykład wektoryzacji znajduje się poniżej:

In [2]:
# PRZYKŁAD ------------------------------------------
# np. Dla dwóch dokumentów:
# Dokument 1: Ala ma kota i ma psa
# Dokument 2: Kot ma Alę

# Poszczególne kroki wyglądają następująco:
# Lista unikalnych słów:                 [Ala, ma, kota, i, psa, Kot, Alę]
# Szablon wektora cech:                  [  0,  0,    0, 0,   0,   0,   0] - wektor jest tyluelementowy, ile mamy  unikalnych słów
# Osadzenie dokumenu 1 jako wektor cech: [  1,  2,    1, 1,   1,   0,   0] - słowo "ma" pojawia się w dok. 2 razy, "kot" i "Alę" - wcale
# Analogicznie dokument 2:               [  0,  1,    0, 0,   0,   1,   1]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(train['text']) # stwórz macierz liczbową z danych. W wierszach mamy kolejne dokumenty, w kolumnach kolejne pola wektora cech odpowiadające unikalnym słowom (bag of words)
X_test_counts = vectorizer.transform(test['text'])       # analogicznie jak wyżej - dla zbioru testowego.

print("Rozmiar stworzonej macierzy: {x}".format(x=X_train_counts.shape)) # wyświetl rozmiar macierzy. Pierwsze pole - liczba dokumentów, drugie - liczba cech (stała dla wszystkich dokumentów)
print("Liczba dokumentów: {x}".format(x=X_train_counts.shape[0]))
print("Rozmiar wektora bag-of-words {x}".format(x=X_train_counts.shape[1]))


Rozmiar stworzonej macierzy: (1624, 37325)
Liczba dokumentów: 1624
Rozmiar wektora bag-of-words 37325


### Uwaga:
Na zbiorze treningowym użyto funkcji - fit_transform(), na testowym - transform(). <br/>
**Dlaczego?** fit_transform() wykonuje dwie operacje - tworzy i zapisuje listę wszystkich unikalnych słów (słownik) oraz zamienia dokument na wektor o długości takiej jak słownik. transform() natomiast wykorzystuje istniejący już słownik i wykonuje z jego użyciem transformację do wektora.
<br/>
Ponieważ zbiór treningowy jest zazwyczaj liczniejszy - z reguły znajdziemy w nim więcej różnych słów. Ponadto, wszystkie słowa, które mogą pomóc w klasyfikacji i tak muszą znaleźć się w zbiorze treningowym aby móc się nauczyć ich wykorzystania. Nie nadpisuje się zatem słownika za pomocą zbioru testowego i tworzy się go tylko raz - podczas treningu, wykorzystując go następnie do tworzenia nowych wektorów z nieobserwowanych podczas treningu dokumentów.

# Zadanie 1 (1 punkt):
Jak się pewnie domyślasz - reprezentacja bag-of-words będzie miała bardzo wiele zer w wygenerowanych macierzach (macierzach, w których w poszczególnych wierszach będziemy mieli poszczególne dokumenty, a w kolumnach wektory słów reprezentacji bag of words). Rozmiar macierzy z poprzedniego listingu pokazuje, że każdy dokument opisany jest wektorem 37325 pozycji, ponieważ tyle różnych słów zostało wykrytych po analizie wszystkich dokumentów treningowych. Większość dokumentów analizowanych osobbno zawierać będzie pewnie co najwyżej kilkadziesiąt/kilkaset różnych słów.

***Zadanie: Napisz fragment kodu, który zliczy:***
<ol>
    <li><strong>jaki procent macierzy X_train_counts ma elementy o wartości różnej od zera</strong></li>
    <li><strong>ile tokenów (łącznie, nie tylko unikalne) występuje w macierzy X_train_counts?</strong></li>
</ol>
Wskazówka - ponieważ zer w tej macierzy jest istotnie dużo - dane po transformacji CountVectorizerem trzymane są w specjalnym formacie, w którym zapisuje się tylko elementy mające wartości różne od zera, w tzw. macierzy rzadkiej (sparse matrix). Aby przeiterować po takiej macierzy, można wykorzystać następujące fragmenty kodu:

<strong>cx = X_train_counts.tocoo()</strong> - transformuj macierz do reprezentacji koordynatowej (patrz niżej) <br/>
<strong>for doc_id, word_id, count in zip(cx.row, cx.col, cx.data):</strong> - pozwala ona na iterowanie po wszystkich niezerowych elementach, w każdym kroku otrzymując 3 zmienne - numer wiersza (numer dokumentu), numer kolumny (identyfikator słowa ze słownika) oraz licznik mówiący ile razy dane słowo wystąpiło w danym dokumencie. <span style="color: #ff0000">(Do wykonania zadania musisz zaktualizować linijki 3, 7 i 8)</span>

In [4]:
count_tokens = 0   # tu zapisz liczbę wszystkich tokenów w macierzy
count_nonzero = 0  # tu zapisz ilość elementów niezerowych w macierzy
count_all = X_train_counts.shape[0]  * X_train_counts.shape[1]  # tu zapisz ilość komórek w macierzy (ilość wierszy * ilość kolumn, rozważ użycie pola 'shape' na macierzy X_train_counts)
print(count_all)
cx = X_train_counts.tocoo()
for doc_id, word_id, count in zip(cx.row, cx.col, cx.data):    #iteracja po elementach niezerowych
    count_tokens += count # + liczba wystapien slowa
    count_nonzero += 1

print("W datasecie znajduje się: {tokens} tokenów. Macierz posiada {nonzero_percent}% elementów niezerowych".format(
    tokens=count_tokens,
    nonzero_percent = round(100.0*count_nonzero/count_all, 3)
))

60615800
W datasecie znajduje się: 435292 tokenów. Macierz posiada 0.394% elementów niezerowych


<div class="alert alert-block alert-success">
    <strong>Oczekiwany rezultat:</strong> <br/>
Mniej niż 1% elementów niezerowych (!) <br/>
Ponad 400000 tokenów
</div>

# Zadanie 2 (1 punkt) - słowa charakteryzujące klasy

Wykorzystajmy macierz X_train_counts wykorzystywaną w poprzednim zadaniu, a także etykiety kategorii, aby stworzyć listy najczęściej występuących słów w danych kategoriach. <br/><br/>
Aby ułatwić zadanie, utworzono większość funkcji **get_top_occuring_words()** tworzącej taki ranking<br/>
**Zadanie 2a (0.5 punktu)**: twoim zadaniem jest zaktualizowanie wartości pola: **category_word_counts[category][word]**, tak, aby poprawnie zliczyć ile razy dane słowo wystąpiło w kategorii. <span style="color: #ff0000">(zaktualizuj linijkę 20)</span>
<br/>
Czy najczęstsze słowa pozwalają rozdzielić kategorie SPAM od HAM?  
**Jeśli najczęstsze słowa z kategori SPAM i HAM byłyby różne to tak. W tym przypadku listy są bardzo podobne więc jest to raczej niemożliwe.**

In [5]:
import operator

def get_top_occuring_words(X_train_counts, how_many_words, vectorizer, train):
    id_to_word = {v: k for k, v in vectorizer.vocabulary_.items()} # stwórz mapowanie pozycji wektora bag-of-words na konkretne słowa
    cx = X_train_counts.tocoo()

    category_word_counts = dict()      # słownik, w którym przeprowadzimy zliczanie

    for doc_id, word_id, count in zip(cx.row, cx.col, cx.data):
        category = train.iloc[doc_id]['label']  # w category znajduje się idetyfikator kategorii dla aktualnego dokumentu, zapisujemy go
        word = id_to_word[word_id]              # w word - aktualne słowo z dokumentu
                                                # mamy też liczność wystąpienia danego słowa w dokumencie (gdzie? :) )

        if category not in category_word_counts.keys(): # stwórzmy słownik z kategoriami jako kluczami
            category_word_counts[category] = dict()     # jeśli widzimy nową kategorię - dodajemy do słownika

        if word not in category_word_counts[category]: # w ramach każdej kategorii będziemy zliaczać słowa
            category_word_counts[category][word] = 0.0 # jeśli aktualne słowo jeszce nie zotało uwzględnione w kategorii - zainicjujmy jego licznik liczbą 0

        category_word_counts[category][word] += count

    for category_name in category_word_counts.keys(): # wyświetl nazwy kategorii i n najczęściej występujących w nich słów
        sorted_cat = sorted(category_word_counts[category_name].items(), key=operator.itemgetter(1), reverse=True) # posortowany dict() słowo -> liczność, wg liczności, malejąco
        print("{cat}: {top}".format(cat=category_name, top=[word for word, count in sorted_cat[:how_many_words]])) # wyświetl nazwę kategorii i top n słów


get_top_occuring_words(X_train_counts, 12, vectorizer, train) # wywołanie funkcji

ham: ['the', 'to', 'of', 'and', 'is', 'in', 'it', 'that', 'for', 'you', 'on', 'with']
spam: ['the', 'of', 'to', 'and', 'in', 'you', 'nbsp', 'for', 'is', 'your', 'this', 'as']


### Wektoryzacja Tf-Idf

Po wykonaniu zadania 2a widzimy, że najczęściej występujące słowa w każdej kategorii mają niewielką użytezczność (pasują do każdej kategorii). Aby sprawić, żeby na czele rankingu znalazły się słowa charakterystyczne dla danej klasy, możemy użyć metody Tf-idf. <br/>
**Zadanie 2b:
Nadpisz wartości X_train_counts oraz X_test_counts wykorzystując w tym celu TfidfVectorizer** (http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) zamiast CountVectorizer, ustaw parametr max_df na 0.4 (tzn. ignoruj słowa, które występują w więcej niż 40% dokumentów). Następnie wykonaj stworzoną w zadaniu 2 funkcję get_top_occuring_words(), aby sprawdzić, czy ranking najważniejszych słów się zmienił. Czy zmieniony zestaw słów lepiej reprezentuje kategorie? <span style="color: #ff0000">(zaktualizuj linie 3, 4, 5)</span>

**W tym przypadku najczęściej występujące słowa dla kategorii SPAM i HAM różnią się co może pomóc w rozróżnieniu tych kategorii.**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.4)
X_train_counts = vectorizer.fit_transform(train['text']) # stwórz macierz wektorów. W wierszach mamy kolejne dokumenty, w kolumnach kolejne pola wektora cech odpowiadające unikalnym słowom
X_test_counts = vectorizer.transform(test['text']) # analogicznie dla zbioru testowego.

get_top_occuring_words(X_train_counts, 12, vectorizer, train) # wywołanie funkcji

ham: ['debian', 'org', 'lists', '20', 'unsubscribe', 'linux', 'net', 'wrote', 'list', 'my', 'can', 'www']
spam: ['nbsp', 'our', '20', 'click', 'here', 'spam', '2009', 'content', 'hibody', 'we', 'free', 'all']


# Zadanie 3 - Stemming i lematyzacja (1 punkt)
Często istotne słowa występują w wielu odmianach (szczególnie w językach fleksyjnych, takich jak nasz), np: university - universities ; pay - paid - paying - pays . Wielość odmian słów ma swoje przełożenie na rozmiar słownika.
<br/><br/>
W niektórych warunkach, w szczególności:
<ul>
<li>Kiedy mamy ograniczoną pamięć</li>
<li>Kiedy ważny jest dla nas czas działania algorytmu</li>
<li>Kiedy istnieje ryzyko przeuczenia</li>
</ul>
warto rozważyć znormalizowanie słów, tak, aby zmniejszyć rozmiar słownika, a co za tym idzie wymagania pamięciowe (a co za tym idzie - czas treningu/klasyfikacji). Ograniczenie rozmiaru słownika może też zapobiec przeuczeniu. Normalizację możemy wykonać np. poprzez zastosowanie stemmingu lub lematyzacji dla poszczególnych wyrazów.
<br/>
<strong>Zadanie 3a (0.5 punktu)</strong>: Z użyciem biblioteki NLTK wykonaj zarówno lematyzację (używając WordNetLemmatizer) jak i stemming (używając PorterStemmer) tekstu zawartego w sample_text. Uwaga - lematyzator opcjonalnie wymaga pos-tagu dla tokenu. Przekaż do funkcji lematyzującej zmienną current_word_postag jako drugi argument. <span style="color: #ff0000">(zaktualizuj linie 20, 21, 34, 35)</span>

<strong>Zadanie 3b (0.5 punktu)</strong>: O ile zmniejszyła się liczba unikalnych słów po zastosowaniu lematyzacji? Odpowiedź zawrzyj w komentarzu. <span style="color: #ff0000">(linijki 43:45)</span>

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

def get_wordnet_pos(treebank_tag):     # lematyzator wymaga, aby dla danego słowa podać mu, czy jest to czasownik, rzeczownik czy inny POS-tag. Funkcja jest adapterem tagów nadanych przez funkcję pos_tag do tagów wymaganych przez lematyzator
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()

sample_text = "There are some cheaper alternatives for buying the red trousers. There is a discount, it is so cheap!"

lemmatized = [] # tutaj będziemy dopisywać zlematyzowane słowa
stemmed = []    # tutaj będziemy dopisywać wystemowane słowa

tokenized = word_tokenize(sample_text)
print("unikalne tokeny: ", len(set(tokenized)))
pos_tokens = nltk.pos_tag(tokenized)       # nadajemy pos-tagi (rzeczownik, czasownik przymiotnik...) każdemu słowu



for i in range(len(tokenized)): # dla każdego słowa
    current_word_postag = get_wordnet_pos(pos_tokens[i][1]) # pobieramy pos-tag słowa
    lemmatized_token = wordnet_lemmatizer.lemmatize(tokenized[i], pos=current_word_postag)
    stemmed_token = porter_stemmer.stem(tokenized[i])

    lemmatized.append(lemmatized_token)
    stemmed.append(stemmed_token)
print("wystemowane", len(set(stemmed)))
print("zlematyzowane: ", len(set(lemmatized)))

print("Bazowy tekst:        {t}".format(t=sample_text))
print("Wystemowany tekst:   {t}".format(t=" ".join(stemmed)))
print("Zlematyzowany tekst: {t}".format(t=" ".join(lemmatized)))

# Ile uniklanych tokenów znajduje się w tekście bazowym?: 19
# Ile unikalnych tokenów znajduje się w tekście wystemowanym?: 19
# Ile unikalnych tokenów w tekście zlematyzowanym?: 17
# Różnica ilości unikalnych tokenów między tekstem bazowym a wystemowanym i bazowym a zlematyzowanym: 0 i 2

unikalne tokeny:  19
wystemowane 19
zlematyzowane:  17
Bazowy tekst:        There are some cheaper alternatives for buying the red trousers. There is a discount, it is so cheap!
Wystemowany tekst:   there are some cheaper altern for buy the red trouser . there is a discount , it is so cheap !
Zlematyzowany tekst: There be some cheap alternative for buy the red trouser . There be a discount , it be so cheap !


---
# Zadanie 4 (1 punkt) - klasyfikacja i interpretacja wyników
Mając już dobrą reprezentację danych i wiedząc jak działa normalizacja - możemy klasyfikować! <br/>
Istnieje wiele algorytmów, które dobrze radzą sobie z klasyfikacją tekstu, kilka przykładów to:
<ul>
<li>Naiwny klasyfikator Bayesa</li>
<li>Maszyna wektorów nośnych - SVM</li>
<li>Sieci neuronowe</li>
</ul>
O sieciach neuronowych więcej powiemy na jednych z przyszłych laboratoriów. <br/>
<strong>Zadanie 4a (0.5 punktu)</strong> Wykorzystując przetworzoną postać danych: X_train_counts, X_test_counts z poprzednich zadań oraz dokumentację sklearn, zaimplementuj klasyfikację z użyciem naiwnego klasyfikatora Bayesa (MultinomialNB). <span style="color: #ff0000">(zaktualizuj linie 7, 9, 12)</span>

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

def labels_as_strings(vector_of_indices): # funkcja pomocnicza zamieniająca identyfikatory numeryczne na tekstowe
    return ['ham' if ind == 0 else 'spam' for ind in vector_of_indices]

nb = MultinomialNB()

nb.fit(X_train_counts, train["label_num"])
predicted = nb.predict(X_test_counts)

print("Ile elementów testowych udało się poprawnie zaklasyfikować?")
accuracy = accuracy_score(test['label_num'], predicted)
print(accuracy)
print("Szczegółowy raport (per klasa)")
print(classification_report(labels_as_strings(test['label_num']), labels_as_strings(nb.predict(X_test_counts)))) # testowanie klasyfikatora - szerokie podsumowanie uwzględniające miary: precision, recall, f1


Ile elementów testowych udało się poprawnie zaklasyfikować?
0.8840381991814461
Szczegółowy raport (per klasa)
              precision    recall  f1-score   support

         ham       0.86      1.00      0.92       517
        spam       1.00      0.61      0.76       216

    accuracy                           0.88       733
   macro avg       0.93      0.80      0.84       733
weighted avg       0.90      0.88      0.87       733



**Zadanie 4b (0.5 punktu)
Po analizie szczegółowego raportu z zadania 4a - odpowiedz na poniższe pytania i zapisz odpowiedzi w komentarzu:**
<ol>
<li>Która miara mówi nam o tym, jak wiele spośród elementów uznanych za spam rzeczywiście jest spamem?  </li>
<li>Która miara mówi nam o tym, jak wiele spośród wszystkich elementów rzeczywiście będących spamem zostało wykrytych jako spam?</li>

<li>Która kategoria została w ogólnym rozrachunku lepiej rozpoznana przez klasyfikator, jeśli zależy nam bardziej na tym, żeby klasyfikator, jeśli mówi, że coś należy do danej klasy, raczej się w tym nie mylił, niż żeby wykrył wszystkie elementy klasy?</li>

</ol>

In [ ]:
# odp zad 4.1: precision
# odp zad 4.2: recall
# odp zad 4.3: spam

Sklearn jest bardzo wdzięcznym narzędziem, w którym proces klasyfikacji możemy wykonać w zaledwie kilku linijkach. Bardzo przydatną klasą jest klasa Pipeline, która definiuje sekwencję kroków, które wykonujemy wywołując metodę fit().
W naszym przypadku mamy dwa kroki:
<ol>
    <li>Wektoryzacja - zamienia dane zapisane w postaci tekstowej na macierz z wektorami bag-of-words.</li>
    <li>Klasyfikacja - wytrenowanie klasyfikatora.</li>
</ol>
W zdefiniowanym obiekcie typu pipeline, i+1 element pipeline'u na wejściu dostaje dane z wyjścia i-tego elementu (Zatem nasz klasyfikator otrzyma dane przetworzone przez TfidfVectorizer). <br/>
Metoda fit na wejściu przyjmuje listę dokumentów w formie tekstowej, oraz oczekiwane etykiety w formie liczbowej.
<br/>
Analogicznie w procesie klasyfikowania nowych tekstów z użyciem istniejącego modelu - metoda predict() wykona sekwencję kroków: wektoryzacja + klasyfikacja dla zadanej listy surowych tekstów). <br/>
Zapoznaj się z poniższym kodem i uruchom go.

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas
import numpy as np

# ------------------- WCZYTANIE DANYCH -----------

full_dataset = pandas.read_csv('spam_emails.csv', encoding='utf-8')      # wczytaj dane z pliku CSV
full_dataset['label_num'] = full_dataset.label.map({'ham':0, 'spam':1})  # ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam", wykonujemy mapowanie tych wartości na liczby, aby móc wykonać klasyfikację.

np.random.seed(0)                                       # ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
train_indices = np.random.rand(len(full_dataset)) < 0.7 # wylosuj 70% wierszy, które znajdą się w zbiorze treningowym

train = full_dataset[train_indices] # wybierz zbior treningowy (70%)
test = full_dataset[~train_indices] # wybierz zbiór testowy (dopełnienie treningowego - 30%)


# ------------------- STWORZENIE PIPELINE'U -----------

pipeline = Pipeline([             # stwórzmy pipeline surowy tekst -> TFIDF vectorizer -> klasyfikator
    ('tfidf', TfidfVectorizer(max_df=0.4)),
    ('clf', MultinomialNB()),
])

# ------------------- TRANSFORMACJA I UCZENIE -----------

pipeline.fit(train['text'], train['label_num']) # zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym

# ------------------- KLASYFIKACJA PRZYKŁADOWEGO TEKSTU -----------

text_to_predict = "NEED TO FIND SOMETHING? ::FREE MORTGAGE QUOTE:: To be removed from this list, click here. "
predicted = pipeline.predict([text_to_predict])
if predicted == 1:
    detected = 'SPAM'
else:
    detected = 'HAM'
print("Tekst {t}, zaklasyfikowany został jako: {d}".format(t=text_to_predict, d=detected))

# ------------------- OCENA KLASYFIKATORA -----------
accuracy = pipeline.score(test['text'], test['label_num'])
print("W zbiorze testowym {n}% przypadków zostało poprawnie zaklasyfikowanych!".format(
    n=100.*accuracy))

Tekst NEED TO FIND SOMETHING? ::FREE MORTGAGE QUOTE:: To be removed from this list, click here. , zaklasyfikowany został jako: SPAM
W zbiorze testowym 88.4038199181446% przypadków zostało poprawnie zaklasyfikowanych!


# Zadanie 5 (1 punkt): dobór parametrów klasyfikacji
Poniżej znajduje się kod tworzący pipeline składający się z dwóch elementów: TfidfVectorizera oraz klasyfikatora naiwnego Bayesa - MultinomialNB. Wektoryzator tworzy model bag-of-words, który uwzględnia jedynie 1000 najważniejszych słów w słowniku. W celu zastosowania stemmingu oraz lematyzatora w treningu i predykcji stoworzona została klasa TheTokenizer, która poza podziałem tekstu na słowa wykonuje również zadania normalizacji wg. ustalonych flag: **use_stemming, use_lemmatization, use_stopword_removal**. <br/>
<strong>Zadanie 5a (0.5 punktu)</strong>: <br/>
Zweryfikuj jak zmiana wartości flag **use_stemming, use_lemmatization, use_stopword_removal**, a co za tym idzie wykorzystanie lamatyzacji, stemmingu i usuwania najczęstszych słow wpływa na miary precision, recall i f1 stworzonego klasyfikatora. Wyniki zapisz w komentarzu. <span style="color: #ff0000">(modyfikuj linie 16, 17, 18, komentarz - w kolejnej komórce)</span><br/>
<strong> Zadanie 5b (0.5 punktu)</strong>: <br/>
Ustaw flagi **use_stemming, use_lemmatization, use_stopword_removal** z linii 16,17 i 18 na False, i porównaj wartości precision recall i f1 dla klasyfikatora, ktory wykorzystuje CountVectorizer i takiego, który wykorzystuje TfidfVectorizer. Pozostaw parametr max_features=1000 w obu przypadkach. Który wektoryzator jest lepszy? <span style="color: #ff0000">(modyfikuj linię 84)</span>
</ol>

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
# wczytywanie danych
from sklearn.datasets import fetch_20newsgroups # zbiór danych zawarty w Sklearn, który zawiera dane z 20 grup newsowych
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import stopwords


class TheTokenizer(object):              # Aby zastosować lematyzację/stemming z NLTK, musimy napisać własny tokenizator, który podzieli tekst na słowa i przekształci je na stemy/lematy.
    def __init__(self):
        self.use_stemming = False         #czy stemować?
        self.use_lemmatization = False    #czy lematyzować?
        self.use_stopword_removal = False  #czy usunąć słowa częste jak the, and, of itp.
        # usuwanie słów częstych wpływa na polepszenie miar, stemming i lematyzacja w połączeniu z usuwaniem częstych słów równierz wpływają na polepszenie miar z tym że stemming wpływa lepiej na precyzje a lematyzacja na recall
        self.wnl = WordNetLemmatizer()   # Utwórz lematyzator oparty na wordnet
        self.stemmer = PorterStemmer()   # Utwórz stemmer Portera
        self.stopwords = set(stopwords.words('english')) # załaduj listę ~100 najczęstszych słów (the, and, of, ...)

    def __call__(self, doc):
        if not self.use_stemming and not self.use_lemmatization: # tokenizuj i ew. lematyzuj/stemuj/usuń stopwords w zależności od ustawionych flag
            return [t for t in word_tokenize(doc) if self.allow(t)]
        elif self.use_stemming and not self.use_lemmatization:
            return [self.stem_token(t) for t in word_tokenize(doc) if self.allow(t)]
        elif self.use_lemmatization and not self.use_stemming:
            return [self.lemmatize_token(t, pos) for t, pos in pos_tag(word_tokenize(doc)) if self.allow(t)]

    def stem_token(self, t):
        return self.stemmer.stem(t)

    def lemmatize_token(self, t, postag):
        return self.wnl.lemmatize(t, self.get_wordnet_pos(postag))

    def allow(self, t):
        if not self.use_stopword_removal:
            return True

        if t in self.stopwords:
            return False
        else:
            return True

    def get_wordnet_pos(self, treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

def labels_as_strings(vector_of_indices): # funkcja pomocnicza zamieniająca identyfikatory numeryczne na tekstowe
    return [dataset_train.target_names[ind] for ind in vector_of_indices]


print("Pobieranie danych")
categories = ['misc.forsale', 'soc.religion.christian', 'sci.space', 'talk.politics.guns',
              'comp.graphics', 'sci.med',  'rec.motorcycles',  'sci.med',
              'sci.electronics', 'talk.politics.misc', 'comp.sys.mac.hardware'] # lista kategorii, które chcemy analizować

dataset_train = fetch_20newsgroups(subset='train',
                                   categories=categories,
                                   shuffle=True,
                                   random_state=42) # pobieramy zbiór uczący (na nim będziemy trenować) dla wybranych kategorii.


dataset_test = fetch_20newsgroups(subset='test',
                                  categories=categories,
                                  shuffle=True,
                                  random_state=42) # pobieramy zbiór testowy (na nim będziemy testować) dla wybranych kategorii



print("Tworzenie pipeline'u")
pipeline = Pipeline([             # stwórzmy pipeline surowy tekst -> vectorizer -> klasyfikator
    ('vectorizer', CountVectorizer(tokenizer=TheTokenizer(), max_features=1000)), # lepszy jest TfidfVectorizer
    ('clf', MultinomialNB()),
])


print("Uczenie pipeline'u")
pipeline.fit(dataset_train.data, dataset_train.target) # trenujemy klasyfikator!


print("Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika?)?")
print("W słowniku znajduje się {n} różnych słów".format(
    n=len(pipeline.named_steps['vectorizer'].vocabulary_.keys())
))


print("Ocena klasyfikatora")
print(classification_report(labels_as_strings(dataset_test.target), labels_as_strings(pipeline.predict(dataset_test.data)))) # testowanie klasyfikatora - szerokie podsumowanie uwzględniające miary: precision, recall, f1

Pobieranie danych
Tworzenie pipeline'u
Uczenie pipeline'u


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Ile różnych słów tworzy wektor bag of words (jaki jest rozmiar słownika?)?
W słowniku znajduje się 1000 różnych słów
Ocena klasyfikatora
                        precision    recall  f1-score   support

         comp.graphics       0.67      0.65      0.66       389
 comp.sys.mac.hardware       0.62      0.66      0.64       385
          misc.forsale       0.72      0.86      0.78       390
       rec.motorcycles       0.60      0.81      0.69       398
       sci.electronics       0.61      0.58      0.60       393
               sci.med       0.77      0.53      0.63       396
             sci.space       0.77      0.69      0.73       394
soc.religion.christian       0.88      0.83      0.86       398
    talk.politics.guns       0.58      0.48      0.52       364
    talk.politics.misc       0.48      0.54      0.51       310

              accuracy                           0.67      3817
             macro avg       0.67      0.66      0.66      3817
          weighted avg       

In [ ]:
Top 1000 - CountVectorizer
      Precision:   Recall:   F1:
      0.68      0.67      0.67
Top 1000 - TfIDF:
    Bez usuwania stopwords:
        Precision:   Recall:   F1:
         0.74      0.72      0.72
    Usuwanie stopwords:
        Precision:   Recall:   F1:
         0.75      0.74      0.74
    Usuwanie stopwords + stemming:
        Precision:   Recall:   F1:
        0.78      0.77      0.77
    Usuwanie stopwords + lematyzacja:
        Precision:   Recall:   F1:
         0.79      0.78      0.77
